In [46]:
import spacy
import csv
import string
import collections
from itertools import groupby

def load_tweets(path='data/TweetData.txt'):
    tweets = []
    with open(path, encoding='ascii', errors='ignore') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ')
        for row in reader:
            if row[-2] == 'en':
                yield ' '.join(row[2:-2])

def preprocess(tweets, abbreviations):
    hashtags = collections.Counter()
    unique_tweets = collections.Counter()
    VALID_CHARACTERS = string.ascii_letters + string.digits + "-.,;:!?'"
    
    for tweet in tweets:
        sentence = []
        
        tweet_chars = (c for c in tweet if c in string.printable)
        for word in (list(group) for k, group in groupby(tweet_chars, lambda x: x == " ") if not k):
            sentence.append(''.join(word))
        
        if sentence[0] == 'RT':
            sentence = sentence[2:]
        
        sentence = [word for word in sentence if word != '-' and not word.startswith('http')]
        sentence = [word[1:].capitalize() if word.startswith('@') else word for word in sentence]
        
        end_sentence = len(sentence) - 1
        for j, word in reversed(list(enumerate(sentence))):
            end_sentence = j
            if word.startswith('#'):
                tmp_hashtag = sentence[j][1:].lower()
                if tmp_hashtag in abbreviations:
                    tmp_hashtag = abbreviations[tmp_hashtag]
                hashtags.update([tmp_hashtag])
            else:
                break
        
        # Remove '#' and '...' at the end
        if end_sentence > 0:
            sentence = sentence[:end_sentence + 1]
            if sentence[-1].endswith('...'):
                sentence = sentence[:-1]
        
        unique_tweets.update([' '.join(
            (''.join([c for c in word if c in VALID_CHARACTERS]) for word in sentence)
        )])
    
    return (unique_tweets, hashtags)
    
[x for x in preprocess(load_tweets(), {})[0]][:100]

['Joedotie: Video:http:t.coAbYDbhTG2q out the future looks promising',
 "Paul Scholes reveals he isn't 'passionate' about coaching via Mailsport",
 'Culture, commitment to customers, brokers and agents have to be project in your brand.',
 'Dont blanket the entire country with a listing only people in your town care about.',
 'Arturo Vidals agent has agreed personal terms with Man United  the midfielder must now make a decision on his future.',
 'A future president of the United States is possibly listening to Justin Bieber right now.',
 'future president of the United States. We have gotten past the bs. And we definitely should hang out more.',
 'Manchester United Target Juan Cuadrado to Decide Future on Today.FTB -AMMAD',
 'Arturo Vidals agent has agreed personal terms with Man United  the midfielder must now make a decision on his future',
 'Giving the consumers to interact with intelligent real estate information.',
 'Manchester United Target Juan Cuadrado to Decide Future on Friday